In [1]:
import random
import requests
import pandas as pd
from pprint import pprint

from faker import Faker
from qdrant_client import QdrantClient, models

In [2]:
client = QdrantClient(url="http://localhost:6333")

In [ ]:
COLLECTION_NAME = "CS-AGENT-TEST"

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE),
)

True

In [36]:
fake = Faker()

orders = []

for i in range(1, 6):
    order = {}
    order["ID"] = i
    order["Customer Name"] = fake.name()
    order["Email"] = fake.email()
    amount_net = round(random.uniform(20.0, 500.0), 2)
    tax = round(amount_net * 0.19, 2)  # Assuming 19% tax
    amount_gross = round(amount_net + tax, 2)
    order["Total Amount Net"] = amount_net
    order["Total Amount Gross"] = amount_gross
    order["Order Date"] = fake.date_time_between(start_date="-1m", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    order["Paid"] = random.choice([True, False])
    orders.append(order)

df = pd.DataFrame(orders)
df.head(10)

,ID,Customer Name,Email,Total Amount Net,Total Amount Gross,Order Date,Paid
0,1,Dr. Rachael Thompson DDS,johnsonjames@example.net,130.20,154.94,2024-12-20 15:18:20,True
1,2,Tabitha Arias,jlong@example.net,367.59,437.43,2024-12-20 15:18:25,False
2,3,Jason Carson,nicholseric@example.org,213.07,253.55,2024-12-20 15:18:28,True
3,4,Alicia Parrish,mandy58@example.org,300.82,357.98,2024-12-20 15:18:58,True
4,5,Robert Gallegos,feliciawatts@example.org,149.16,177.50,2024-12-20 15:18:45,True


In [37]:
ID = 1548
Customer_Name = "Paul Müller"
Email = "paulm@example.com"
TotalAmountNet = 100.0
TotalAmountGross = 119.0
OrderDate = "2024-12-20 14:21:23"
Paid = False

In [38]:
new_order = pd.DataFrame([{
    "ID": ID,
    "Customer Name": Customer_Name,
    "Email": Email,
    "Total Amount Net": TotalAmountNet,
    "Total Amount Gross": TotalAmountGross,
    "Order Date": OrderDate,
    "Paid": Paid,
}])

df = pd.concat([df, new_order], ignore_index=True)

In [39]:
df.head(10)

,ID,Customer Name,Email,Total Amount Net,Total Amount Gross,Order Date,Paid
0,1,Dr. Rachael Thompson DDS,johnsonjames@example.net,130.20,154.94,2024-12-20 15:18:20,True
1,2,Tabitha Arias,jlong@example.net,367.59,437.43,2024-12-20 15:18:25,False
2,3,Jason Carson,nicholseric@example.org,213.07,253.55,2024-12-20 15:18:28,True
3,4,Alicia Parrish,mandy58@example.org,300.82,357.98,2024-12-20 15:18:58,True
4,5,Robert Gallegos,feliciawatts@example.org,149.16,177.50,2024-12-20 15:18:45,True
5,1548,Paul Müller,paulm@example.com,100.00,119.00,2024-12-20 14:21:23,False


In [40]:
# Inserting data into the collection
points = []
for i, row in df.iterrows():
    point = models.PointStruct(
        id=row["ID"],
        payload=row.to_dict(),
        vector=[random.random() for _ in range(100)],
    )
    points.append(point)

client.upsert(
    collection_name=COLLECTION_NAME,
    points=points,
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
x = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=1,
    with_payload=True,
    with_vectors=False,
)

pprint(x[0][0].payload)

{'Customer Name': 'Dr. Rachael Thompson DDS',
 'Email': 'johnsonjames@example.net',
 'ID': 1,
 'Order Date': '2024-12-20 15:18:20',
 'Paid': True,
 'Total Amount Gross': 154.94,
 'Total Amount Net': 130.2}


In [42]:
# Search for the order by email
x = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=[random.random() for _ in range(100)],
    limit=1,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="Email", match=models.MatchValue(value=x[0][0].payload["Email"])
            )
        ]
    ),
    with_payload=True,
    with_vectors=False,
)

pprint(x[0].payload)

{'Customer Name': 'Dr. Rachael Thompson DDS',
 'Email': 'johnsonjames@example.net',
 'ID': 1,
 'Order Date': '2024-12-20 15:18:20',
 'Paid': True,
 'Total Amount Gross': 154.94,
 'Total Amount Net': 130.2}
